<a id="TOP"></a>
<img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=150/><img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=200/> 

***

# NASA ISRO Synthetic Aperture Radar Mission
## Query and Download from ASF DAAC (disabled until NISAR data on ASF)
## Preprocess NISAR-simulated ALOS2 products 
#### This notebook uses a stack of GCOV products that were created with the notebook: Ecosystems_InputPrep_ALOS2-NISAR.ipynb. The files were uploaded to the scratch space. 

Authors: Alex Christensen


## 1 &emsp; Import Python Modules

In [ ]:
import os
from pathlib import Path

import sys

# notebook_dir = Path(os.getcwd())
notebook_dir = Path('/scratch/alex_eco_test/')
main_dir = notebook_dir.resolve().parents[0]


aoi = 'little_river'
aoi_dir = notebook_dir/aoi
Path(aoi).mkdir(parents=True,exist_ok=True)

print(notebook_dir)
import rasterio.mask


In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from osgeo import gdal, osr, ogr
import subprocess

import time
import scipy
import pandas as pd
from pandas import DataFrame
from IPython.display import Image
import sklearn  # imported from scikit-learn
from sklearn import metrics
from matplotlib.pyplot import figure
from matplotlib.colors import ListedColormap
%matplotlib inline
import rasterio

# from ipywidgets import interactive
from rasterio.plot import show_hist

# import asf_search as asf
import geopandas as gpd
from shapely.geometry import shape, Polygon

# from ipyleaflet import (
#     Map,  basemaps,
#     Rectangle,
#     GeoJSON,
#     DrawControl,GeoData
# )
# import ipyleaflet
import h5py

In [ ]:
input_dir = aoi_dir / 'GCOV'
output_dir = aoi_dir / 'GCOV_stacks'
# ancillary_dir = main_dir / 'ancillary_data'
Path(input_dir).mkdir(parents=True, exist_ok=True)
Path(output_dir).mkdir(parents=True, exist_ok=True)

print(output_dir)


## 2:  Set the Query Parameters

## 2.1: Define the AOI
Provide either a geojson defining the AOI or draw one on the map

In [ ]:
# m = Map(basemap=basemaps.Esri.WorldImagery, center=(0,0), zoom=2)
# # poly_color = '#00F'
# draw_control = DrawControl()

# draw_control.rectangle = {"shapeOptions": {"fillColor": "#fca45d", "color": "#fca45d", "fillOpacity": 1.0 }}

# def clear_m():
#     global rects,polys
#     rects = set()
#     polys = set()

# clear_m()
# def handle_draw(self, action, geo_json):
#     global rects,polys
#     polygon=[]
#     for coords in geo_json['geometry']['coordinates'][0][:-1][:]:
#         polygon.append(tuple(coords))
#     polygon = tuple(polygon)
#     if geo_json['properties']['style']['color'] == '#00F':  # poly
#         if action == 'created':
#             polys.add(polygon)
#         elif action == 'edited':
#             #polys.update(polygon)
#             polys.clear()
#             polys.add(polygon)
#         elif action == 'deleted':
#             polys.discard(polygon)

# draw_control.on_draw(handle_draw)
# m.add_control(draw_control)
  
# m

In [ ]:
# if os.path.isfile('%s.geojson' %(aoi_inputs/aoi))==False:
#     polygon: Polygon = shape(draw_control.last_draw.get('geometry'))
#     polygon.geom_type
#     print(polygon)
#     poly_df = gpd.GeoDataFrame(geometry=[polygon],crs='epsg:4326')
#     poly_df.to_file('%s.geojson' %(aoi_inputs/aoi),driver='GeoJSON')
# else:
#     gdf = gpd.read_file('%s.geojson' %(aoi_inputs/aoi))
#     polygon = gdf.geometry[0]

# wkt = str(polygon)


## 2.2: Define the search query for NISAR data
Options (more options and details are available here: https://docs.asf.alaska.edu/api/keywords/)
- platform
- instrument
- frame
- processinglevel
- flightDirection
- start
- end
- maxResults

In [ ]:
# opts = {
#     'platform':asf.PLATFORM.SENTINEL1,
#     'processingLevel': asf.PRODUCT_TYPE.GRD_HD,
#     'flightDirection':asf.FLIGHT_DIRECTION.ASCENDING,
#     'start':'Feb 12, 2017',
#     'end':'December 11, 2017',
#     # 'maxResults':5
#     }

## 2.3: Submit search and decide whether or not to download


In [ ]:
# results = asf.geo_search(intersectsWith=wkt, **opts)
# print(f'{len(results)} results found')


## 2.4: Download files.
You must enter your ASF/EarthData login credentials

In [ ]:
# ## Change to True if you want to download all of the results. Otherwise, adjust search query before continuing
# download = False
# if download:
#     session = asf.ASFSession().auth_with_creds('achri', '**')
#     results.download(path=aoi_dir, session=session)
# else:
#     print('Not downloading yet')

## 3: Preprocess GCOV products

## 3.1: Find, unzip, and open HDF5 files

In [ ]:
# from s3path import PureS3Path
# import s3fs
# import glob
# import os
# import h5py
# s3 = s3fs.S3FileSystem(anon=False)

# # s3.glob(indir + 'LED-ALOS2*1.1__*')

In [ ]:
h5_files = glob.glob(str(input_dir/ '*gcov_*.h5'))
# h5_files = s3.glob(str(aoi_inputs/ 'GCOV*.h5'))
print(*h5_files,sep='\n')


In [ ]:
## clean up previously processed geotiffs
clean = True
if clean:
    old_files = glob.glob('%s/*.tif' %(output_dir))
    for i in range(len(old_files)):
        os.system('rm -r %s' %(old_files[i]))


In [ ]:
f = h5py.File(h5_files[0], "r") 
a_group_key = list(f.keys())[0]
f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA'].keys()

In [ ]:
for i in range(len(h5_files)):
    filename = h5_files[i].split('/')[-1].split('_')
    startdate = filename[-1][:-4]
    # enddate = filename[-6]
    
    # f = h5py.File(s3.open(h5_files[i], "rb"))
    f = h5py.File(h5_files[i], "r") 
    a_group_key = list(f.keys())[0]
    ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
    ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object
    # ds_epsg = f[a_group_key]['LSAR']['GCOV']['metadata']['radarGrid']['epsg']
    ds_epsg = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['projection'][()]
    ds_HHHH = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HHHH'][()]  # returns as a numpy array
    ds_HVHV = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HVHV'][()]  # returns as a numpy array
    
    print(h5_files[i].split('/')[-1])
    # print('Dates: ', startdate, ' - ',enddate)
    print('Raster bounds: ',min(ds_x),max(ds_x),min(ds_y),max(ds_y))
    print('X Size: ',ds_x.shape[0],' Y Size: ',ds_y.shape[0])
    print('Resolution: ', ds_y[0] - ds_y[1],'m')
    print('')
    meta = {'driver': 'GTiff', 
            'dtype': 'float32', 
            'nodata': None, 
            'width': ds_x.shape[0], 
            'height': ds_y.shape[0], 
            'count': 1, 
            'crs': rasterio.CRS.from_epsg(ds_epsg[()]), 
            'transform': rasterio.Affine(ds_x[1] - ds_x[0], 0.0, ds_x[0], 0.0, ds_y[1] - ds_y[0], ds_y[0]), 
            'tiled': False, 
            'interleave': 'band'}
    fig,[ax1,ax2] = plt.subplots(ncols=2)
    ax1.imshow(ds_HHHH,vmin=0,vmax=0.2)
    ax1.set_title('HHHH')
    ax2.imshow(ds_HVHV,vmin=0,vmax=0.1)
    ax2.set_title('HVHV')
    plt.show()
    with rasterio.open('%s_HHHH.tif' %(output_dir/h5_files[i].split('/')[-1][:-3]), 'w', **meta) as dst:
            dst.write(ds_HHHH,indexes=1)    
    with rasterio.open('%s_HVHV.tif' %(output_dir/h5_files[i].split('/')[-1][:-3]), 'w', **meta) as dst:
        dst.write(ds_HVHV,indexes=1)    

    

## 3.2: Crop reference image and resample/coregister all images in stack

In [ ]:
crop = False
if crop:
    crop_to = gpd.read_file(glob.glob(str(aoi_dir/ '*.geojson'))[0])
    crop_to = crop_to.to_crs(rasterio.CRS.from_epsg(ds_epsg[()]))

    crop_to.plot()
    crop_to = crop_to.explode()
    shapes = crop_to.geometry
    
   

## 3.3: Get Target Resolution and Target Extent from the Reference (first image in the stack)

If you want to change the resolution, set ref_tr manually


In [ ]:
tif_files = glob.glob('%s/*.tif' %(output_dir))
tif_files = [item for item in tif_files if 'subset' not in item]
if crop:
    with rasterio.open(tif_files[0]) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=crop)
        ref_meta = src.meta

    ref_meta.update({"driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform})
else:
    with rasterio.open(tif_files[0]) as src:
        out_image = src.read()
        out_transform = src.transform
        ref_meta = src.meta
        
with rasterio.open('%s_subset.tif' %(tif_files[0][:-4]), "w", **ref_meta) as dest:
    dest.write(out_image)

with rasterio.open('%s_subset.tif' %(tif_files[0][:-4])) as src:
    ref_te = src.bounds
    ref_tr = out_transform[0]

# ref_tr = 100

In [ ]:
for i in range(len(tif_files)):
    os.system('gdalwarp -overwrite -tr %s %s -te %s %s %s %s -tap -srcnodata nan -dstnodata nan %s %s_subset_coreg.tif' %(ref_tr,ref_tr,ref_te.left,ref_te.bottom,ref_te.right,ref_te.top,tif_files[i],tif_files[i][:-4]))
    # os.system('gdalwarp -overwrite -tr %s %s -te %s %s %s %s -tap -srcnodata nan -dstnodata nan %s_HVHV.tif %s_HVHV_subset_coreg.tif'  %(ref_tr,ref_tr,ref_te.left,ref_te.bottom,ref_te.right,ref_te.top,h5_files[i][:-4],h5_files[i][:-4]))


## 4: Clean up extra files

In [ ]:
for i in range(len(tif_files)):
    if i==0:
        os.system('rm -r %s_subset.tif' %(tif_files[i][:-4]))
    os.system('rm -r %s.tif' %(tif_files[i][:-4]))


## 5: Plot an example

In [ ]:
HH = rasterio.open('%s_subset_coreg.tif' %(tif_files[0][:-4])).read(1)
HV = rasterio.open('%s_subset_coreg.tif' %(tif_files[1][:-4])).read(1)

fig, [ax1,ax2] = plt.subplots(1,2,figsize=(20,40))
im1 = ax1.imshow(HH,vmin=0,vmax=.05)
im2 = ax2.imshow(HV,vmin=0,vmax=.02)
ax1.set_title('HHHH')
ax2.set_title('HVHV')

divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im1, cax=cax, orientation='vertical')

divider = make_axes_locatable(ax2)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im2, cax=cax, orientation='vertical');